In [1]:
from faker import Faker
import pandas as pd
import random
import dash
import datetime
import re
import plotly.graph_objs as go
from dash import dcc, html
from dash.dependencies import Input, Output

# Configurar o pandas para exibir todas as linhas e colunas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Lista de data frames criados

In [14]:
whos

Variable                Type         Data/Info
----------------------------------------------
Faker                   type         <class 'faker.proxy.Faker'>
Input                   type         <class 'dash.dependencies.Input'>
Output                  type         <class 'dash.dependencies.Output'>
aluno_df                DataFrame        id              nome\<...>n19  20       Alexis Lara
app                     Dash         <dash.dash.Dash object at 0x00000251199B5520>
dash                    module       <module 'dash' from 'c:\\<...>ages\\dash\\__init__.py'>
datetime                module       <module 'datetime' from '<...>hon39\\lib\\datetime.py'>
dcc                     module       <module 'dash.dcc' from '<...>\dash\\dcc\\__init__.py'>
disciplina_df           DataFrame       id codigo        nome\<...>n4   5    QUI     Química
go                      module       <module 'plotly.graph_obj<...>graph_objs\\__init__.py'>
grafico_layout          Div          Div([Graph(id='grafi

# Criando Tabelas com dados fakes

In [2]:
# [OPCIONAL] Importar cada DataFrame ja criado
professor_df = pd.read_csv('professor_df.csv')
aluno_df =pd.read_csv('aluno_df.csv')
disciplina_df = pd.read_csv('disciplina_df.csv')
turma_df = pd.read_csv('turma_df.csv')
notas_df = pd.read_csv('notas_df.csv')
media_final_df = pd.read_csv('media_final_df.csv')

## Tabelas: Professor, Aluno e Disciplina

In [ ]:
fake = Faker()

# Criando a tabela Professor
num_professores = 5
professor_data = {
    'id': range(1, num_professores + 1),
    'nome': [fake.name() for _ in range(num_professores)],
    'email': [fake.email() for _ in range(num_professores)]
}
professor_df = pd.DataFrame(professor_data)

# Criando a tabela Aluno
num_alunos = 20
aluno_data = {
    'id': range(1, num_alunos + 1),
    'nome': [fake.name() for _ in range(num_alunos)]
}
aluno_df = pd.DataFrame(aluno_data)

# Criando a tabela Disciplina
num_dis = 5
disciplina_data = {
    'id': range(1, num_dis + 1),
    'codigo': ['MAT', 'FIS', 'BIO', 'HIS', 'QUI'],
    'nome': ['Matemática', 'Física', 'Biologia', 'História', 'Química']
}
disciplina_df = pd.DataFrame(disciplina_data)


## Tabela: Turma

In [ ]:
fake = Faker()

# Criando a tabela Turma
turma_data = []
id = 1

# Preencher os dados da turma para cada professor
for index, row in professor_df.iterrows():
    professor_id = row['id']
    disciplina_id = index + 1

    # Selecionar aleatoriamente 10 alunos para cada disciplina
    alunos_ids = random.sample(range(1, num_alunos + 1), 10)
    status = random.choice(['Em andamento', 'Notas pendentes', 'Finalizada'])  # Variando o status
    
    for aluno_id in alunos_ids:
        turma_id = int(f"{professor_id}{disciplina_id}")
        
        # Primeira prova (entre março e abril)
        turma_data.append({
            'id': id,
            'prof_disc_id': turma_id,
            'professor_id': professor_id,
            'disciplina_id': disciplina_id,
            'aluno_id': aluno_id,
            'started_at': fake.date_time_between_dates(datetime_start=pd.to_datetime('2023-03-01'), 
                                                       datetime_end=pd.to_datetime('2023-04-30')).strftime('%Y-%m-%d'),
            'tipo': 'Semestral',
            'status': 'Finalizada',
            'logica_nota_final': '(P1+P2)/2',
            'media_final_minima': 5.0
        })
        id += 1

        # Segunda prova (entre maio e junho) e variamos o status
        turma_data.append({
            'id': id,
            'prof_disc_id': turma_id,
            'professor_id': professor_id,
            'disciplina_id': disciplina_id,
            'aluno_id': aluno_id,
            'started_at': fake.date_time_between_dates(datetime_start=pd.to_datetime('2023-05-01'), 
                                                       datetime_end=pd.to_datetime('2023-06-30')).strftime('%Y-%m-%d'),
            'tipo': 'Semestral',
            'status': status,
            'logica_nota_final': '(P1+P2)/2',
            'media_final_minima': 5.0
        })
        id += 1

turma_df = pd.DataFrame(turma_data)

In [3]:
# Calcular a distribuição do status por disciplina apenas para os dados filtrados
print(turma_df.query("started_at > '2023-04-30'").groupby(['disciplina_id', 'status']).size().unstack(fill_value=0))

status         Em andamento  Finalizada  Notas pendentes
disciplina_id                                           
1                         0           0               10
2                         0          10                0
3                         0          10                0
4                        10           0                0
5                         0          10                0


## Tabela: Nota

In [ ]:
# Filtrar os dados da tabela turma_df para incluir apenas os casos em que o status é "Finalizada"
turma_finalizada = turma_df[turma_df['status'] == 'Finalizada']

# Criando a tabela Notas
notas_data = []

for index, row in turma_finalizada.iterrows():
    turma_id = row['id']
    aluno_id = row['aluno_id']
    started_at = datetime.datetime.strptime(row['started_at'], '%Y-%m-%d')
    
    # Verificar o tipo de nota com base no started_at
    if started_at.month in [3, 4]:
        tipo_nota = 'P1'
    elif started_at.month in [5, 6]:
        tipo_nota = 'P2'
    else:
        continue  # Ignorar se o started_at estiver fora dos períodos desejados
    
    # Gerar notas para cada tipo (P1, P2)
    nota = round(random.uniform(1, 10), 1)  # Gerando nota aleatória entre 1 e 10
    notas_data.append({
        'turma_id': turma_id,
        'aluno_id': aluno_id,
        'nota': nota,
        'aval': tipo_nota
    })

notas_df = pd.DataFrame(notas_data)


## Tabela: Media_Final

In [ ]:
# Criando a tabela de notas finais por disciplina
media_final_data = []

# Iterar sobre cada disciplina na tabela de turma
for disciplina_id in turma_df['disciplina_id'].unique():
    # Filtrar as notas correspondentes à disciplina na tabela de notas
    notas_disciplina = notas_df.merge(turma_df, left_on=['turma_id','aluno_id'], right_on=['id','aluno_id'])
    notas_disciplina = notas_disciplina[(notas_disciplina['disciplina_id'] == disciplina_id)]

    # Iterar sobre cada aluno na disciplina
    for aluno_id in notas_disciplina['aluno_id'].unique():
        # Obter a lógica de notas finais para a disciplina atual
        logica_nota_final = turma_df.loc[turma_df['disciplina_id'] == disciplina_id, 'logica_nota_final'].iloc[0]
        media_final_minima = turma_df.loc[turma_df['disciplina_id'] == disciplina_id, 'media_final_minima'].iloc[0]

        # Usar expressão regular para encontrar todas as variáveis na lógica
        variaveis = re.findall(r'\b\w+\b', logica_nota_final)
        variaveis = [var for var in variaveis if var[0].isalpha()]

        # Inicializar um dicionário para armazenar as notas correspondentes a cada variável
        notas_aluno = {}

        # Iterar sobre as variáveis e buscar as notas correspondentes nas observações de avaliação
        for var in variaveis:
            nota = notas_disciplina[(notas_disciplina['aluno_id'] == aluno_id) & (notas_disciplina['aval'] == var)]['nota'].values
            if nota.size > 0:
                notas_aluno[var] = nota[0]

        # Substituir as variáveis na lógica pelas notas correspondentes e calcular a nota final
        try:
            # Substituir as variáveis na string de lógica pelo valor correspondente
            for var, val in notas_aluno.items():
                logica_nota_final = logica_nota_final.replace(var, str(val))
            nota_final = eval(logica_nota_final)
            
            # Adicionar os dados à tabela de notas finais por disciplina
            media_final_data.append({
                'disciplina_id': disciplina_id,
                'aluno_id': aluno_id,
                'nota_final': nota_final,
                'aprovado': nota_final >= media_final_minima
            })
        except:
            print(f'Não foi possível calcular a nota final para o aluno {aluno_id} na disciplina {disciplina_id}.')

# Criar DataFrame para a tabela de notas finais por disciplina
media_final_df = pd.DataFrame(media_final_data)


## Exemplo

In [4]:
turma_df.loc[turma_df['aluno_id'] == 13]

,id,professor_id,disciplina_id,aluno_id,started_at,tipo,status,logica_nota_final,media_final_minima
0,1,1,1,13,2023-04-28,Semestral,Finalizada,(P1+P2)/2,5.0
1,2,1,1,13,2023-05-30,Semestral,Notas pendentes,(P1+P2)/2,5.0
74,75,4,4,13,2023-04-17,Semestral,Finalizada,(P1+P2)/2,5.0
75,76,4,4,13,2023-05-31,Semestral,Em andamento,(P1+P2)/2,5.0
94,95,5,5,13,2023-04-14,Semestral,Finalizada,(P1+P2)/2,5.0
95,96,5,5,13,2023-05-15,Semestral,Finalizada,(P1+P2)/2,5.0


In [5]:
notas_df.loc[notas_df['aluno_id'] == 13]

,turma_id,aluno_id,nota,aval
0,1,13,9.3,P1
57,75,13,9.3,P1
74,95,13,7.3,P1
75,96,13,3.5,P2


In [6]:
media_final_df.loc[media_final_df['aluno_id'] == 13]

,disciplina_id,aluno_id,nota_final,aprovado
27,5,13,5.4,True


# Graficos

### Parte 1: Definição dos dados

In [7]:
# Tabelas fornecidas
# professor_df, aluno_df, disciplina_df, turma_df, notas_df

# Agrupando as notas por turma e aluno
notas_aluno_agrupadas = notas_df.groupby(['turma_id', 'aluno_id']).agg(media_final=('nota', 'mean')).reset_index()

# Juntando as tabelas turma_df e notas_aluno_agrupadas para obter a média das notas por turma
media_notas_turma = turma_df.merge(notas_aluno_agrupadas, left_on=['id', 'aluno_id'], right_on=['turma_id', 'aluno_id']).groupby('turma_id')['media_final'].mean()

# Calcular a média total das notas
media_total = media_notas_turma.mean()

# Contar o número de disciplinas e alunos
num_disciplinas = len(disciplina_df)
num_alunos = len(aluno_df)



### Parte 2: Criação dos KPIs

In [8]:
# Inicializar a aplicação Dash
app = dash.Dash(__name__)

# Layout dos KPIs
kpis_layout = html.Div([
    html.Div([
        html.Div([
            html.H2('Média Total das Notas'),
            html.H3(id='media-total-notas')
        ], className='kpis'),
        html.Div([
            html.H2('Número de Disciplinas'),
            html.H3(id='num-disciplinas')
        ], className='kpis'),
        html.Div([
            html.H2('Número de Alunos'),
            html.H3(id='num-alunos')
        ], className='kpis')
    ], className='container', id='kpis-container')  # Verifique se o ID 'kpis-container' está definido corretamente aqui
])


### Parte 3: Definição do Gráfico de Barras

In [9]:
# Layout do Gráfico de Barras
grafico_layout = html.Div([
    dcc.Graph(id='grafico-media-notas')
])

### Parte 4: Definição das callbacks

In [10]:
# Callback para atualizar o gráfico com os dados da média das notas por turma
@app.callback(
    Output('grafico-media-notas', 'figure'),
    [Input('dropdown-turma', 'value')]
)
def update_graph(selected_turma_ids):
    if not selected_turma_ids:  # Se nenhuma turma for selecionada, não faça nada
        return go.Figure()

    # Filtrar o DataFrame media_notas_turma com base nas turmas selecionadas
    media_notas_turma_filtrado = media_notas_turma[media_notas_turma.index.isin(selected_turma_ids)]

    # Criar o gráfico de barras
    data = []
    tickvals = []
    ticktext = []
    for i, (turma_id, media) in enumerate(media_notas_turma_filtrado.items()):
        trace = go.Bar(
            x=[i],  # Usar um índice como valor no eixo X
            y=[media],  # Média das notas
            name=f'Turma {turma_id}'
        )
        data.append(trace)
        tickvals.append(i)  # Adicionar o índice como valor de tick
        ticktext.append(f'Turma {turma_id}')  # Adicionar o nome da turma como rótulo do tick

    layout = go.Layout(
        title='Média das Notas por Turma',
        xaxis=dict(
            title='Turma ID',
            tickvals=tickvals,  # Valores dos ticks no eixo X
            ticktext=ticktext  # Rótulos dos ticks no eixo X
        ),
        yaxis=dict(title='Média das Notas')
    )

    fig = go.Figure(data=data, layout=layout)

    return fig

In [11]:
# Callback para atualizar os KPIs com base nas turmas selecionadas
@app.callback(
    Output('kpis-container', 'children'),
    [Input('dropdown-turma', 'value')]
)
def update_kpis(selected_turma_ids):
    if not selected_turma_ids:  # Se nenhuma turma for selecionada, não faça nada
        return [
            html.Div([
                html.H2('Média Total das Notas'),
                html.H3(f'{media_total:.2f}')  # <- Verifique se media_total está definido corretamente
            ], className='kpis'),
            html.Div([
                html.H2('Número de Disciplinas'),
                html.H3(f'{num_disciplinas}')  # <- Verifique se num_disciplinas está definido corretamente
            ], className='kpis'),
            html.Div([
                html.H2('Número de Alunos'),
                html.H3(f'{num_alunos}')  # <- Verifique se num_alunos está definido corretamente
            ], className='kpis')
        ]

    # Filtrar os DataFrames com base nas turmas selecionadas
    turmas_selecionadas = turma_df[turma_df['id'].isin(selected_turma_ids)]
    alunos_selecionados = aluno_df[aluno_df['id'].isin(turmas_selecionadas['aluno_id'].unique())]  # Corrigido aqui
    notas_turmas_selecionadas = notas_aluno_agrupadas[notas_aluno_agrupadas['turma_id'].isin(selected_turma_ids)]

    # Calcular as novas médias e contagens
    nova_media_total = notas_turmas_selecionadas['media_final'].mean()  # <- Aqui estava o problema, verifique se a coluna 'media_final' existe em turmas_selecionadas
    nova_num_disciplinas = len(turmas_selecionadas['disciplina_id'].unique())
    nova_num_alunos = len(alunos_selecionados)

    # Atualizar os KPIs com os novos valores
    return [
        html.Div([
            html.H2('Média Total das Notas'),
            html.H3(f'{nova_media_total:.2f}')
        ], className='kpis'),
        html.Div([
            html.H2('Número de Disciplinas'),
            html.H3(f'{nova_num_disciplinas}')
        ], className='kpis'),
        html.Div([
            html.H2('Número de Alunos'),
            html.H3(f'{nova_num_alunos}')
        ], className='kpis')
    ]


### Parte 5: Componente Dropdown para Seleção de Turmas

In [12]:
# Layout da aplicação
app.layout = html.Div([
    # KPIs
    kpis_layout,
    
    # Gráfico de barras
    grafico_layout,
    
    # Dropdown para seleção de turmas
    dcc.Dropdown(
        id='dropdown-turma',
        options=[{'label': f'Turma {turma_id}', 'value': turma_id} for turma_id in turma_df['id'].unique()],
        multi=True,
        value=[]
    )
])

### Parte 6: Layout Completo e Execução da Aplicação Dash

In [13]:
# Layout completo da aplicação
app.layout = html.Div([
    # Dropdown para seleção de turmas
    dcc.Dropdown(
        id='dropdown-turma',
        options=[{'label': f'Turma {turma_id}', 'value': turma_id} for turma_id in turma_df['id'].unique()],
        multi=True,
        value=[]
    ),

    # KPIs
    kpis_layout,
    
    # Gráfico de barras
    grafico_layout
])

# Executar a aplicação
if __name__ == '__main__':
    app.run_server(debug=True)